<a href="https://colab.research.google.com/github/soyoung33500/ASL/blob/master/%EB%AA%A8%EB%8D%B8_h5_%EB%B9%84%EA%B5%90.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import sklearn
import cv2
from tensorflow import keras
import time
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
from tqdm import tqdm

In [ ]:
# 이미지의 가로 세로
IMG_SIZE = 224
# 비디오에서 학습할 프레임 개수
MAX_SEQ_LENGTH = 20

In [ ]:
# 비디오를 구성하는 이미지의 가운데 부분을 리턴
# frame : 비디오를 구성하는 이미지
def crop_center_square(frame):
    # frame.shape[0:2] : 이미지의 세로, 가로 리턴
    y, x = frame.shape[0:2]
    # 이미지의 세로 가로 중에서 작은 값을 리턴
    min_dim = min(y, x)
    # 이미지의 왼쪽 모서리 좌표의 가로 시작점
    # 이미지 가로 좌표 (이미지 가로 //2  - min_dim//2) ~ (이미지 가로 //2  + min_dim//2)
    start_x = (x // 2) - (min_dim // 2)
    # 이미지의 왼쪽 모서리 세로 좌표 시작점
    # 이미지 세로 좌표 (이미지 세로 //2  - min_dim//2) ~ (이미지 세로 //2  + min_dim//2)
    start_y = (y // 2) - (min_dim // 2)
    return frame[start_y : start_y + min_dim, start_x : start_x + min_dim]

In [ ]:
# 비디오 파일을 읽어서 각 프레임을 이미지로 변환해서 리턴
# path : 비디오 파일 경로
# max_frames : 이미지로 변환할 프레임수
# resize=(IMG_SIZE, IMG_SIZE) : 이미지의 가로 세로
def load_video(path, max_frames=20, resize=(IMG_SIZE, IMG_SIZE)):
    # 비디오 파일을 읽어서 이미지로 변환 할 객페
    cap = cv2.VideoCapture(path)
    frames = []
    try:
        while True:
            # cap.read() : 비디오를 읽어서 리턴
            # ret : 비디오 읽기가 성공했으면 True, 더이상 읽을 비디오 프레임이 없으면 False 가 리턴
            # frame : 비디오 프레임 이미지를 리턴
            ret, frame = cap.read()
            # ret 가 False면 반복 종료
            if not ret:
                break
            # 비디오 이미지를 가운데 리턴
            frame = crop_center_square(frame)
            # 비디오 이미지의 가로 세로를 resize=(224,224) 로 변환
            frame = cv2.resize(frame, resize)
            # frame 은  [줄, 칸, B G R ] 로 구성되 있음
            # frame [ : (모든줄), : (모든칸), R (인덱스2) G (인덱스1) B (인덱스0)] 리턴
            frame = frame[:, :, [2, 1, 0]]

            # frame을 frames에 추가
            frames.append(frame)
        # (20, 224, 224, 3) 인 0으로 초기화된 배열 생성
        frames_arr = np.zeros((max_frames,IMG_SIZE, IMG_SIZE, 3))

        # len(frames) : 전체 프레임수를 20으로 나눈 몫
        # 전체 프레임 중에서 increase_num 씩 증가해서 리턴
        increase_num = len(frames) // max_frames
        # 0~max_frames 까지 반복
        for index in range(0, max_frames):
            # index가 len(frames) (전체 프레임수) 이상이면 반복 종료
            if index >= len(frames):
                break
            # frames_arr에 frame[index * increase_num] 대입
            frames_arr[index] = frames[index * increase_num]

    finally:
        # 비디오 이미지 변환 종료
        cap.release()
    # frames_arr 를 np.uint8 타입 numpy 배열로 변환 해서 리턴
    return np.array(frames_arr, dtype=np.uint8)

In [ ]:
# 모든 비디오 파일의 이미지와 종류를 리턴
# video_name : 비디오 파일명
# label: 비디오 파일 종류
# root_dir : 비디오 파일 경로
def prepare_all_videos(video_name , label, root_dir, is_train):
    # 비디오 파일 이름의 개수
    num_samples = len(video_name)
    # video_name.values.tolist() : 비디오 파일 이름을 리스트로 변환 해서 리턴
    video_paths = video_name.values.tolist()
    # 비디오 종류를 리턴
    labels = label.values
    # 비디오 종류를 2차원 배열로 변환
    labels = labels.reshape(-1, 1)
    # 0으로 초기화된 [비디오 파일개수 , 20 , 224 , 224, 3] 배열 생성
    x = np.zeros(shape = (num_samples, MAX_SEQ_LENGTH, IMG_SIZE, IMG_SIZE, 3), dtype = "float32" )
    # 0으로 초기화된 [비디오파일개수] 인 배열 생성
    y = np.zeros(shape = (num_samples), dtype="float32")

    # 비디오 파일의 개수 만큼 반복
    for index, path in tqdm( enumerate(video_paths) ):
        # 비디오 파일을 읽어서 비디오 이미지를 리턴
        video = load_video(root_dir + path)
        # 학습데이터 일때 이미지 증강 (테스트 데이터는 증강 안함)
        if is_train:
            # video에 저장된 20개의 이미지를 증강
            video = seq(images = video)

        # video를 x에 추가
        x[index] = np.array(video / 255, dtype="float32")
        # 비디오의 종류가 저장된 labels의 index 번째를 y에 추가
        y[index]= np.array(labels[index], dtype="float32")

    # 전체 비디오 프레임을 이미지로 변환한 x
    # to_categorical(y) : 이미지의 종류 y 를 onehot 인코딩
    return (x, to_categorical(y))

# 새로운 테스트 영상

In [ ]:
# 데이터 경로 (경로 마지막 부분에 / 꼭 넣기)
data_path = "/content/drive/MyDrive/비디오 분류_1조/01.video_classification/dataset/"
# 테스트 데이터의 파일명과 종류가 저장된 test.csv 를 읽음
test_df = pd.read_csv(data_path + "test.csv")
# label 컬럼에 저장된 데이터를 중복을 제거하고 조회
# text_label에 저장 (shuffle 하기 전에 저장)
text_label = test_df["tag"].unique()

In [ ]:
# 스포츠 종류 라벨 확인
# 0 : 'CricketShot'
# 1 : 'Punch'
# 2 : 'TennisSwing'
text_label

array(['CricketShot', 'Punch', 'TennisSwing'], dtype=object)

In [ ]:
# 해당 경로에 있는 .mp4 파일명 리스트 가져오기
import os
path = '/content/drive/MyDrive/비디오 분류_1조/new_test/'
file_list_dir = os.listdir(path)
# 파일명 끝이 .mp4인 경우 file_list 에 추가
file_list = [file for file in file_list_dir if file.endswith('.mp4')]

In [ ]:
len(file_list)

70

In [ ]:
# label을 입력할 빈 리스트 생성
label = []
for i in range(len(file_list)):
    # 파일명에 'cricket'이 있으면 label에 0 입력
    if 'cricket' in file_list[i]:
        label.append(0)
    # 파일명에 'punch'이 있으면 label에 1 입력
    elif 'punch' in file_list[i]:
        label.append(1)
    # 파일명에 'tennis'이 있으면 label에 2 입력
    elif 'tennis' in file_list[i]:
        label.append(2)
    # 위의 것 중 어느 것에도 해당되지 않으면 잘못된 파일명
    else:
        print('잘못된 파일명')

In [ ]:
# video_name 에 동영상 파일 이름이 저장돼있는 file_list 입력
# 해당하는 스포츠 종류의 index가 저장돼있는 label 입력
new = pd.DataFrame({"video_name" : file_list, "label" : label})
new

,video_name,label
0,cricketshot021.mp4,0
1,cricketshot022.mp4,0
2,cricketshot023.mp4,0
3,punch021.mp4,1
4,punch022.mp4,1
...,...,...
65,punch005.mp4,1
66,punch009.mp4,1
67,punch006.mp4,1
68,punch008.mp4,1


In [ ]:
arr = list(new['label'].value_counts())
arr

[24, 23, 23]

In [ ]:
new['label'].value_counts()

1    24
2    23
0    23
Name: label, dtype: int64

In [ ]:
pd.DataFrame({"video" : ['CricketShot', 'Punch', 'TennisSwing'], "count" : [23, 24, 23]})

In [ ]:
# sklearn.utils.shuffle(new) : new에 저장된 행을 섞음
new = sklearn.utils.shuffle(new)
# new에 저장된 줄이 섞였음
new

,video_name,label
41,punch020.mp4,1
32,tennisswing007.mp4,2
3,punch021.mp4,1
5,punch023.mp4,1
66,punch009.mp4,1
...,...,...
56,cricketshot020.mp4,0
37,tennisswing002.mp4,2
18,cricketshot002.mp4,0
28,tennisswing009.mp4,2


In [ ]:
# 모든 비디오 파일의 이미지와 종류를 리턴
# prepare_all_videos(video_name , label, root_dir, is_train)
# video_name : 비디오 파일명
# label: 비디오 파일 종류
# root_dir : 비디오 파일 경로, 마지막에 꼭 / 를 붙여줘야 합니다.
# is_train : train 데이터 여부 (test 데이터이므로 여기서는 False)
new_X_test, new_y_test = prepare_all_videos(new["video_name"],
                                            new["label"] ,
                                            "/content/drive/MyDrive/비디오 분류_1조/new_test/",
                                            False)

8it [00:16,  2.12s/it]


KeyboardInterrupt: ignored

# 여러 모델 한번에 확인

In [ ]:
models = ['M', 'X', 'I', 'R', 'i3d']
acc_list = []
loss_list = []

for i in models:
    print("i=", i)
    if i == "X" :
        new_X_test_small=np.zeros(shape = (len(new_X_test), 10, 224,224,3), dtype = 'float32')
        for index in range(len(new_X_test)):
            for j in range(0,20,2):
                new_X_test_small[index] = new_X_test[index, j, :, :, :]

        print(new_X_test_small.shape)

    else:
        print('no')



## 기본증강 + LSTM

In [ ]:
# 모델 이름 저장된 list
models = ['M', 'X', 'I', 'R', 'i3d']
acc_list = []
loss_list = []

for i in models:
    best_model = keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/result/" + i + "result01.h5")
    if i == "X" or i == "I":
        new_X_test_small=np.zeros(shape = (len(new_X_test), 10, 224,224,3), dtype = 'float32')
        for index in range(len(new_X_test)):
            for j in range(0,20,2):
                new_X_test_small[index] = new_X_test[index, j, :, :, :]
        loss, acc = best_model.evaluate(new_X_test_small, new_y_test, batch_size = 1)
    else:
        loss, acc = best_model.evaluate(new_X_test, new_y_test, batch_size = 1)
    loss_list.append(round(loss, 4))
    acc_list.append(round(acc, 4))
    print("Model :", i, "loss :", loss, "acc :", acc)
    # best_model = None
    # time.sleep(2)
model_name = ['MobileNet', 'Xception', 'Inception', 'ResNet', 'i3d']
df01 = {'Model' : model_name, 'loss' : loss_list, 'acc' : acc_list}
df01

In [ ]:
plt.figure(figsize=(10, 5))
plt.bar(df01['Model'], df01['acc'])
for i in range(0, 5):
        plt.text(i, df01['acc'][i]+0.05, df01['acc'][i], ha="center", fontsize=15)
plt.ylim(0, 1.1)
plt.xticks(fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.bar(df01['Model'], df01['loss'], color='green')
for i in range(0, 5):
        plt.text(i, df01['loss'][i]+0.05, df01['loss'][i], ha="center", fontsize=15)
plt.ylim(0, 1.1)
plt.xticks(fontsize=15)
plt.show()

---

## 바꾼증강 + LSTM

In [ ]:
# 모델 이름 저장된 list
models = ['M', 'X', 'I', 'R', 'i3d']
acc_list = []
loss_list = []

for i in models:
    best_model = keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/result/" + i + "result02.h5")
    if i == "X" or i == "I":
        new_X_test_small=np.zeros(shape = (len(new_X_test), 10, 224,224,3), dtype = 'float32')
        for index in range(len(new_X_test)):
            for j in range(0,20,2):
                new_X_test_small[index] = new_X_test[index, j, :, :, :]
        loss, acc = best_model.evaluate(new_X_test_small, new_y_test, batch_size = 1)
    else:
        loss, acc = best_model.evaluate(new_X_test, new_y_test, batch_size = 1)
    loss_list.append(round(loss, 4))
    acc_list.append(round(acc, 4))
    print("Model :", i, "loss :", loss, "acc :", acc)
model_name = ['MobileNet', 'Xception', 'Inception', 'ResNet', 'i3d']
df02 = {'Model' : model_name, 'loss' : loss_list, 'acc' : acc_list}
df02

In [ ]:
plt.figure(figsize=(10, 5))
plt.bar(df02['Model'], df02['acc'])
for i in range(0, 5):
        plt.text(i, df02['acc'][i]+0.05, df02['acc'][i], ha="center", fontsize=15)
plt.ylim(0, 1.1)
plt.xticks(fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(10, 5))
plt.bar(df02['Model'], df02['loss'], color='green')
for i in range(0, 5):
        plt.text(i, df02['loss'][i]+0.05, df02['loss'][i], ha="center", fontsize=15)
plt.ylim(0, 1.3)
plt.xticks(fontsize=15)
plt.show()

---

## 기본증강 + GRU

In [ ]:
# 모델 이름 저장된 list
models = ['M', 'X', 'I', 'R']
acc_list = []
loss_list = []

for i in models:
    best_model = keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/result/" + i + "result03.h5")
    if i == "X" or i == "I":
        new_X_test_small=np.zeros(shape = (len(new_X_test), 10, 224,224,3), dtype = 'float32')
        for index in range(len(new_X_test)):
            for j in range(0,20,2):
                new_X_test_small[index] = new_X_test[index, j, :, :, :]
        loss, acc = best_model.evaluate(new_X_test_small, new_y_test, batch_size = 1)
    else:
        loss, acc = best_model.evaluate(new_X_test, new_y_test, batch_size = 1)
    loss_list.append(round(loss, 4))
    acc_list.append(round(acc, 4))
    print("Model :", i, "loss :", loss, "acc :", acc)
model_name = ['MobileNet', 'Xception', 'Inception', 'ResNet']
df03 = {'Model' : model_name, 'loss' : loss_list, 'acc' : acc_list}
df03

In [ ]:
plt.figure(figsize=(7, 5))
plt.bar(df03['Model'], df03['acc'])
for i in range(0, 4):
        plt.text(i, df03['acc'][i]+0.05, df03['acc'][i], ha="center", fontsize=15)
plt.ylim(0, 1.1)
plt.xticks(fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(7, 5))
plt.bar(df03['Model'], df03['loss'], color='green')
for i in range(0, 4):
        plt.text(i, df03['loss'][i]+0.05, df03['loss'][i], ha="center", fontsize=15)
plt.ylim(0, 1.0)
plt.xticks(fontsize=15)
plt.show()

---

## 바꾼증강 + GRU

In [ ]:
# 모델 이름 저장된 list
models = ['M', 'X', 'I', 'R']
acc_list = []
loss_list = []

for i in models:
    best_model = keras.models.load_model("/content/drive/MyDrive/Colab Notebooks/result/" + i + "result04.h5")
    if i == "X" or i == "I":
        new_X_test_small=np.zeros(shape = (len(new_X_test), 10, 224,224,3), dtype = 'float32')
        for index in range(len(new_X_test)):
            for j in range(0,20,2):
                new_X_test_small[index] = new_X_test[index, j, :, :, :]
        loss, acc = best_model.evaluate(new_X_test_small, new_y_test, batch_size = 1)
    else:
        loss, acc = best_model.evaluate(new_X_test, new_y_test, batch_size = 1)
    loss_list.append(round(loss, 4))
    acc_list.append(round(acc, 4))
    print("Model :", i, "loss :", loss, "acc :", acc)
model_name = ['MobileNet', 'Xception', 'Inception', 'ResNet']
df04 = {'Model' : model_name, 'loss' : loss_list, 'acc' : acc_list}
df04

In [ ]:
plt.figure(figsize=(7, 5))
plt.bar(df04['Model'], df04['acc'])
for i in range(0, 4):
        plt.text(i, df04['acc'][i]+0.05, df04['acc'][i], ha="center", fontsize=15)
plt.ylim(0, 1.1)
plt.xticks(fontsize=15)
plt.show()

In [ ]:
plt.figure(figsize=(7, 5))
plt.bar(df04['Model'], df04['loss'], color='green')
for i in range(0, 4):
        plt.text(i, df04['loss'][i]+0.05, df04['loss'][i], ha="center", fontsize=15)
plt.ylim(0, 1.3)
plt.xticks(fontsize=15)
plt.show()

---